## Make inference custom container

In [29]:
import os
os.environ['inference_container_name']= "bert2tweet_inference"

In [30]:
%%sh
cd inference_container

# The name of our algorithm
algorithm_name=$inference_container_name


account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

echo $region

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Get the login command from ECR in order to pull down the Tensorflow-gpu:1.5 image
# $(aws ecr get-login --registry-ids 520713654638 --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} . --build-arg REGION=${region}
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

us-west-2
Login Succeeded
Sending build context to Docker daemon  6.656kB
Step 1/32 : FROM ubuntu:18.04
18.04: Pulling from library/ubuntu
a1125296b23d: Pulling fs layer
3c742a4a0f38: Pulling fs layer
4c5ea3b32996: Pulling fs layer
1b4be91ead68: Pulling fs layer
1b4be91ead68: Waiting
4c5ea3b32996: Verifying Checksum
4c5ea3b32996: Download complete
3c742a4a0f38: Verifying Checksum
3c742a4a0f38: Download complete
a1125296b23d: Verifying Checksum
a1125296b23d: Download complete
1b4be91ead68: Verifying Checksum
1b4be91ead68: Download complete
a1125296b23d: Pull complete
3c742a4a0f38: Pull complete
4c5ea3b32996: Pull complete
1b4be91ead68: Pull complete
Digest: sha256:e5b0b89c846690afe2ce325ac6c6bc3d686219cfa82166fc75c812c1011f0803
Status: Downloaded newer image for ubuntu:18.04
 ---> d27b9ffc5667
Step 2/32 : LABEL maintainer="Amazon AI"
 ---> Running in 42985b745049
Removing intermediate container 42985b745049
 ---> a403313a4ffd
Step 3/32 : LABEL com.amazonaws.sagemaker.capabilities.accept

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

COPY failed: stat /var/lib/docker/tmp/docker-builder986662579/sagemaker: no such file or directory
Error response from daemon: No such image: bert2tweet_inference:latest
An image does not exist locally with the tag: 057716757052.dkr.ecr.us-west-2.amazonaws.com/bert2tweet_inference


CalledProcessError: Command 'b'cd inference_container\n\n# The name of our algorithm\nalgorithm_name=$inference_container_name\n\n\naccount=$(aws sts get-caller-identity --query Account --output text)\n\n# Get the region defined in the current configuration (default to us-west-2 if none defined)\nregion=$(aws configure get region)\nregion=${region:-us-west-2}\n\necho $region\n\nfullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"\n\n# If the repository doesn\'t exist in ECR, create it.\n\naws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1\n\nif [ $? -ne 0 ]\nthen\n    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null\nfi\n\n# Get the login command from ECR and execute it directly\n$(aws ecr get-login --region ${region} --no-include-email)\n\n# Get the login command from ECR in order to pull down the Tensorflow-gpu:1.5 image\n# $(aws ecr get-login --registry-ids 520713654638 --region ${region} --no-in

## Remove alias

In [12]:
def normalize_text(s):
    # Desc: retrun cleaned text
    s = s.lower()

    # remove punctuation that is not word-internal (e.g., hyphens, apostrophes)
    s = re.sub('\s\W',' ',s)
    s = re.sub('\W\s',' ',s)

    # make sure we didn't introduce any double spaces
    s = re.sub('\s+',' ',s)
    
    s = re.sub('@[a-zA-Z0-9-]',' ',s)
    

    return s


In [20]:
s = 'Moon-    gon           soo. @kissidea '
k = normalize_text(s)
k

'moon gon soo kissidea '

## Test TweetUtilm

In [3]:
import pandas as pd
import re
import boto3
import numpy as np
import os

from TweetData import TweetData # Custom class for handling input data

In [4]:
file_name = os.path.join('data', "emojis.csv")
def read_data(file_name):
    all_tweets = pd.read_csv(file_name, names=['content', 'sentiment'])
    return all_tweets

all_tweets = read_data(file_name)
print("all_tweets shape: ", all_tweets.shape)

all_tweets shape:  (806204, 2)


In [5]:
def drop_duplicate_records(all_tweets):
    all_tweets_cleaned_df = all_tweets.drop_duplicates() 
    return all_tweets_cleaned_df

all_tweets_cleaned_df = drop_duplicate_records(all_tweets)
print("Cleaned Tweets: \n {} ".format(all_tweets_cleaned_df.shape))
num_tweets = all_tweets_cleaned_df.shape[0]

Cleaned Tweets: 
 (613060, 2) 


In [6]:
num_threshold_lables = 9000 # process more than 9000 for lables
num_use_records = num_tweets # total of tweets

def filter_less_label(all_tweets, num_use_records,   num_filter_less_label):
    # Desc: filter the total data for being less label with specific number
    use_tweets = all_tweets[0:num_use_records]
    tweets = use_tweets.groupby('sentiment').filter(lambda c:len(c) > num_filter_less_label)

    return tweets

tweets = filter_less_label(all_tweets, num_use_records,   num_threshold_lables)
tweets.sentiment.value_counts()

😂    93116
❤    32949
😍    30933
😭    27420
😊    14822
🙄    13473
😩    12348
🔥    11901
🤔    11365
💕     9405
Name: sentiment, dtype: int64

In [7]:
tweet_data = TweetData(tweets)

In [8]:
# make emoji_to_idx and save it into data
emoji_to_idx = tweet_data.make_sentimet_label()

data/emoji_to_idx.pickle is saved
{'❤': 0, '💕': 1, '🔥': 2, '😂': 3, '😊': 4, '😍': 5, '😩': 6, '😭': 7, '🙄': 8, '🤔': 9}


In [9]:
emoji_to_idx

{'❤': 0,
 '💕': 1,
 '🔥': 2,
 '😂': 3,
 '😊': 4,
 '😍': 5,
 '😩': 6,
 '😭': 7,
 '🙄': 8,
 '🤔': 9}

In [10]:
import pickle 

with open('data/emoji_to_idx.pickle', 'rb') as handle:
    emoji_from_disk = pickle.load(handle)
print(emoji_from_disk)

{'❤': 0, '💕': 1, '🔥': 2, '😂': 3, '😊': 4, '😍': 5, '😩': 6, '😭': 7, '🙄': 8, '🤔': 9}


In [11]:
from TweetUtil import TweetUtil

tweet_util = TweetUtil()
tweet_util.load_emoji_data('emoji_to_idx.pickle')
emoji = tweet_util.get_emo_class_label(3)
print(emoji)

emoji_to_idx is loaded
😂


## Delete S3 file

In [ ]:
%store -r

In [ ]:
! aws s3 ls {s3_destination_path_csv}/

In [ ]:
! aws s3 rm {s3_destination_path_csv}/tweet_file_test.csv

## TFDistilBertForSequenceClassification

In [ ]:
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
import tensorflow as tf

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')



In [ ]:
model.summary()

In [ ]:
inputs = tokenizer("Hello, my dog is cute", return_tensors="tf")
inputs["labels"] = tf.reshape(tf.constant(1), (-1, 1)) # Batch size 1

outputs = model(inputs)
loss, logits = outputs[:2]